This file attempts to analyze predict parabola for model on 1% data

In [1]:
import os
import codecs
import json
import spacy
import numpy as np
import pandas as pd
import itertools as it
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from textblob import TextBlob

No handlers could be found for logger "gensim.models.word2vec"


In [2]:
data_directory = os.path.join('dataset')
review_filepath = os.path.join(data_directory,'review2.json')
result_directory = os.path.join('result-3categories-minidata-v2')
review_txt_filepath = os.path.join(result_directory,'review_text.txt')
star_txt_filepath = os.path.join(result_directory,'star_text.txt')

In [3]:
trigram_model_filepath = os.path.join(result_directory,'trigram_model_all')

In [4]:
trigram_sentences_filepath = os.path.join(result_directory,'trigram_sentences_all.txt')

In [5]:
trigram_reviews_filepath = os.path.join(result_directory,'trigram_transformed_reviews_all.txt')

In [6]:
bigram_reviews_filepath = os.path.join(result_directory,'bigram_transformed_reviews_all.txt')

In [7]:
X_list=pd.read_table(bigram_reviews_filepath, skip_blank_lines=False ,names=['text'])
X=X_list.text

In [7]:
y_list=pd.read_table(star_txt_filepath,names=['stars'])
y=y_list.stars

In [8]:
print X.shape
print y.shape

(17396,)
(17396,)


In [9]:
y.value_counts()

3    10403
2     3989
1     3004
Name: stars, dtype: int64

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1)

In [11]:
vect = CountVectorizer()
X_train_dtm = vect.fit_transform(X_train.values.astype('U'))
X_test_dtm = vect.transform(X_test.values.astype('U'))
# X_test_dtm2 = vect.transform(["ddgdhrrrgh00"])

In [12]:
X_test_dtm.shape

(4349, 24921)

In [13]:
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_nb = nb.predict(X_test_dtm)

print metrics.accuracy_score(y_test, y_pred_nb)
#print y_pred_class

0.69671188779


In [68]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.cross_validation import cross_val_score
ab = AdaBoostClassifier(nb)
print cross_val_score(ab, X_train_dtm, y_train, cv=5, n_jobs=-1).mean()

0.680539437687


In [53]:
print metrics.classification_report(y_test, y_pred_nb)
print metrics.confusion_matrix(y_test, y_pred_nb)

             precision    recall  f1-score   support

          1       0.68      0.51      0.58       751
          2       0.45      0.38      0.41       997
          3       0.77      0.87      0.82      2601

avg / total       0.68      0.70      0.68      4349

[[ 382  207  162]
 [ 107  380  510]
 [  73  260 2268]]


In [54]:
y_list.stars.value_counts()/len(y)

3    0.598011
2    0.229306
1    0.172683
Name: stars, dtype: float64

In [55]:
nb.predict_proba(X_test_dtm)
Y_pp = pd.DataFrame(nb.predict_proba(X_test_dtm), columns=['class_1_pp','class_2_pp','class_3_pp'])
print(Y_pp.iloc[0:10])

     class_1_pp  class_2_pp  class_3_pp
0  7.859356e-04    0.998989    0.000225
1  9.995532e-01    0.000441    0.000005
2  1.726834e-01    0.229325    0.597992
3  1.880227e-01    0.669729    0.142248
4  2.330785e-01    0.764005    0.002916
5  1.847622e-07    0.048546    0.951454
6  5.738048e-04    0.024471    0.974956
7  1.989535e-04    0.216245    0.783556
8  3.885253e-04    0.001818    0.997794
9  1.870092e-05    0.006411    0.993570


In [60]:
Y_pred = nb.predict(X_test_dtm)
Y_pp['pred_class_thresh50'] = Y_pred
print(Y_pp.iloc[30:40])

      class_1_pp  class_2_pp    class_3_pp  pred_class_thresh50
30  9.812745e-01    0.018726  2.212150e-08                    1
31  8.897453e-07    0.000010  9.999887e-01                    3
32  8.317781e-01    0.071776  9.644626e-02                    1
33  5.493179e-11    0.015038  9.849616e-01                    3
34  1.395265e-07    0.013913  9.860868e-01                    3
35  7.156852e-02    0.729552  1.988796e-01                    2
36  5.776518e-03    0.002583  9.916403e-01                    3
37  1.189769e-07    0.631364  3.686359e-01                    2
38  1.726834e-01    0.229325  5.979919e-01                    3
39  7.921647e-05    0.000161  9.997594e-01                    3


In [63]:
conmat = np.array(metrics.confusion_matrix(y_test, Y_pp['pred_class_thresh50']))

confusion = pd.DataFrame(conmat, index=['is_1', 'is_2', 'is_3'],
                         columns=['predicted_1','predicted_2', 'predicted_3'])

print(confusion)

      predicted_1  predicted_2  predicted_3
is_1          382          207          162
is_2          107          380          510
is_3           73          260         2268


In [62]:
Y_pp['pred_class_thresh_adj'] = [2 if x >= 0.40 else 0 for x in Y_pp.class_2_pp.values]

conmat2 = np.array(metrics.confusion_matrix(y_test, Y_pp['pred_class_thresh_adj']))

conmat2

array([[   0,    0,    0,    0],
       [ 531,    0,  220,    0],
       [ 581,    0,  416,    0],
       [2274,    0,  327,    0]])

In [64]:
tfi=TfidfVectorizer()
X_train_tfi = tfi.fit_transform(X_train.values.astype('U'))
X_test_tfi = tfi.transform(X_test.values.astype('U'))

In [66]:
nbtfi = MultinomialNB()
nbtfi.fit(X_train_tfi, y_train)
y_pred_nb_tfi = nb.predict(X_test_tfi)

print metrics.accuracy_score(y_test, y_pred_nb_tfi)

0.667279834445


In [16]:
vect = CountVectorizer(ngram_range=(1, 2), min_df=2)
X_train_dtm = vect.fit_transform(X_train.values.astype('U'))
X_test_dtm = vect.transform(X_test.values.astype('U'))
# X_test_dtm2 = vect.transform(["ddgdhrrrgh00"])

In [17]:
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_nb = nb.predict(X_test_dtm)

print metrics.accuracy_score(y_test, y_pred_nb)
#print y_pred_class

0.706139342378
